# Project(실습)
#### 뉴스 수집
- news.naver.com에서 다음의 카테고리별 뉴스 수집
    - 정치, 경제, 사회, 생활/문화, 세계, IT/과학
- 파일이름: 카테고리-고유번호.txr

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.75 Safari/537.36"}
url = "https://news.naver.com/main/main.nhn"
params = {"mode":"LSD",
          "mid":"shm",
          "sid1":100}
categories = {"정치":100, "경제":101, "사회":102, "생활 문화":103, "세계":104, "IT 과학":105}

In [3]:
resp = requests.request("get", url, params=params, headers=headers)
dom = BeautifulSoup(resp.text, "lxml")

In [4]:
# 정치 카테고리의 하이라이트 기사만 수집
# 제목과 url만 뽑아보자.

for _ in dom.select(".cluster_text > a"):
        print(_.text)
        print(_["href"])

미래 먹거리 콘텐츠산업 육성…2022년까지 정책금융 1조 확대(종합)
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=001&aid=0011085852
文 “한류 생산유발 효과 20조…창작자 노력에 날개 달겠다”
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=081&aid=0003029429
文대통령 "콘텐츠는 미래 먹거리…마음껏 상상하고 도전하라"
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=001&aid=0011085158
"BTS·나영석 같은 창작자가 콘텐츠 강국 만들어"
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=006&aid=0000098764
"김정은 눈치 보는 정권" "젊은 청년 두 번 죽여”… 野, 하재헌 중사 공상 판정 맹비난
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=022&aid=0003397964
유승민 “북한의 보훈처냐…온 나라가 미쳐가고 있다”
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=005&aid=0001240086
하재헌 중사 ‘공상’ 논란 정치권 확산…野 “관련자 문책해야”
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=081&aid=0003029417
“대통령 정상 아니라서”… ‘북한 지뢰’ 공상 논란 목소리 높인 野
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=469&aid=0000422831
유성엽, 인사 온 조국에 “내려놓는 게 좋지

In [5]:
# 실습파일 저장용 폴더 생성

import os

if not os.path.exists("8일차_실습_project"):
    os.mkdir("8일차_실습_project")
if not os.path.exists("8일차_실습_project/헤드라인"):
    os.mkdir("8일차_실습_project/헤드라인")
if not os.path.exists("8일차_실습_project/일반기사"):
    os.mkdir("8일차_실습_project/일반기사")

In [6]:
# 하이라이트 기사만 수집

headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.75 Safari/537.36"}
url = "https://news.naver.com/main/main.nhn"
params = {"mode":"LSD",
          "mid":"shm",
          "sid1":None}
categories = {"정치":100, "경제":101, "사회":102, "생활 문화":103, "세계":104, "IT 과학":105}

# catecory마다 크로울링하는 반복문
for category, num in categories.items():
    params["sid1"] = num
    resp = requests.request("get", url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, "lxml")
    print(category, len(dom.select(".cluster_text > a")))
    
    # 헤드라인 기사를 추출하는 반복문
    for _ in dom.select(".cluster_text > a"):
#         print(_.text)
#         print(_["href"])
        
        # 헤드라인 기사 제목과 url만 저장하는 txt 파일.
        with open("8일차_실습_project/헤드라인/"+"{}.txt".format(category), "a", encoding="utf-8") as fp:
            fp.write(",".join([_.text, _["href"]]) + "\n")
        
        # 개별 기사 내용 수집
        articleResp = requests.request("get", _["href"], headers=headers)
        articleDom = BeautifulSoup(articleResp.text, "lxml")
        body = articleDom.select("#articleBodyContents")[0].text.strip()
        articleID = _["href"].split("=")[-1]
        
        # 개별 기사의 본문을 저장하는 txt파일
        with open("8일차_실습_project/헤드라인/"+"{}-{}.txt".format(category, articleID), "w", encoding="utf-8") as fp:
            fp.write(body)

정치 30
경제 22
사회 28
생활 문화 20
세계 24
IT 과학 20


---

# selenium 이용하기

In [7]:
from selenium import webdriver
driver = webdriver.Chrome()

In [8]:
url = "https://news.naver.com"
driver.get(url)

In [9]:
# 카테고리 태그 찾기

for _ in driver.find_elements_by_css_selector(".tx")[2:8]:
    print(_.text)

정치
경제
사회
생활/문화
세계
IT/과학


In [10]:
# 카테고리 클릭

driver.find_elements_by_css_selector(".tx")[2].click()

In [11]:
dom = BeautifulSoup(driver.page_source, "lxml")

In [12]:
for _ in [_.find_next_sibling().select_one("a") for _ in dom.select("#section_body .photo")]:
    print(_.text)
    print(requests.compat.urljoin(url, _["href"]))

‘공정·사상·단식’ 앞세워 조국 투쟁 이어가는 한국당
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=018&aid=0004469721
[단독]서훈 국정원장 극비 해외출장, 북미대화 지원차 미국 갔나
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=081&aid=0003029443
격려·당부·사퇴요구…치열했던 '조국 국회 예방'
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=006&aid=0000098767
북한 지뢰에 두 다리 잃은 예비역 군인, 보훈처 “‘전상’(戰傷)이 아닌 ‘공상’(公傷)” 판정에 분노
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=022&aid=0003397977
민주당 총선대비 물갈이 탄력…양정철·백원우 불출마 의사
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=009&aid=0004428772
조국 "수사 공보준칙 개정이 나 때문? 오해다"
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=047&aid=0002240354
유성엽 "자진사퇴 깊이 생각해야", 조국 "형사절차 통해 진위 밝혀질 것"
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=123&aid=0002212005
[순간포착]“수사공보준칙 개정 부적절…왜? 조국이 하니까”
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=449&aid=0000177817
與 "민생 방치말라"·野 "조국 파면하라"…정기국회

In [13]:
driver.close()

In [14]:
# 헤드라인 이외의 기사만 수집

driver = webdriver.Chrome()
url = "https://news.naver.com"
driver.get(url)

for catnum in range(2,8):
    category = driver.find_elements_by_css_selector(".tx")[catnum].text.replace("/", " ")
    driver.find_elements_by_css_selector(".tx")[catnum].click()
    dom = BeautifulSoup(driver.page_source, "lxml")
    articles = [_.find_next_sibling().select_one("a") for _ in dom.select("#section_body .photo")]
    print(category, len(articles))
    
    for _ in articles:
        # 기사의 제목과 url만 저장하는 txt 파일
        with open("8일차_실습_project/일반기사/"+category+" 일반기사.txt", "a", encoding="utf-8") as fp:
            fp.write(
                ",".join(
                    [
                        _.text, 
                        requests.compat.urljoin(url, _["href"])
                    ]
                ) + "\n")

        # 개별 기사 내용 수집
        articleResp = requests.request("get", 
                                       requests.compat.urljoin(url, _["href"]), 
                                       headers=headers)
        articleDom = BeautifulSoup(articleResp.text, "lxml")
        body = articleDom.select("#articleBodyContents")[0].text.strip()
        articleID = _["href"].split("=")[-1]

        # 개별 기사의 본문을 저장하는 txt파일
        with open("8일차_실습_project/일반기사/"+"{}-{}.txt".format(category, articleID), "w", encoding="utf-8") as fp:
            fp.write(body)
            
driver.close()

정치 20
경제 18
사회 20
생활 문화 19
세계 20
IT 과학 19


---

# 기사 더보기
- 기사 더보기를 눌러서 추가되는 기사를 스크랩해보자

In [22]:
driver = webdriver.Chrome()
driver.get("https://news.naver.com")
driver.find_elements_by_css_selector(".tx")[2].click() # 정치 카테고리로 이동

In [23]:
# 더보기 클릭하기 전 dom을 만들자.
# 더보기 클릭하기 전 기사 갯수

before = BeautifulSoup(driver.page_source, "lxml")
len(before.select("div.cluster_text a"))

56

In [24]:
# 더보기 클릭

print(driver.find_element_by_css_selector(".cluster_more_inner").text)
driver.find_element_by_css_selector(".cluster_more_inner").click()

헤드라인 더보기


> selenium은 브라우저가 페이지를 렌더링하는 시간이 걸린다.
>
> 페이지 렌더링하는 속도보다 코드 실행하는 속도가 더 빠르기 때문에 연달아 코드를 실행하면 문제가 생길 수 있다.
>
> 연달아 실행할 때는 time.sleep(5)

In [25]:
# 더보기 클릭한 후 dom
# 더보기 클릭한 후 기사 갯수가 늘었다.

after = BeautifulSoup(driver.page_source, "lxml")
len(after.select("div.cluster_text a"))

69

In [26]:
# 더보기 클릭 전 기사 목록

for _ in before.select("div.cluster_text a"):
    print(_.text)

미래 먹거리 콘텐츠산업 육성…2022년까지 정책금융 1조 확대(종합)
30이상+
200이상+
文 “한류 생산유발 효과 20조…창작자 노력에 날개 달겠다”
1,600이상+
文대통령 "콘텐츠는 미래 먹거리…마음껏 상상하고 도전하라"
200이상+
"BTS·나영석 같은 창작자가 콘텐츠 강국 만들어"
"김정은 눈치 보는 정권" "젊은 청년 두 번 죽여”… 野, 하재헌 중사 공상 판정 맹비난
500이상+
300이상+
유승민 “북한의 보훈처냐…온 나라가 미쳐가고 있다”
400이상+
하재헌 중사 ‘공상’ 논란 정치권 확산…野 “관련자 문책해야”
300이상+
“대통령 정상 아니라서”… ‘북한 지뢰’ 공상 논란 목소리 높인 野
유성엽, 인사 온 조국에 “내려놓는 게 좋지 않겠나”…사퇴 요구
50이상+
30이상+
"검찰개혁"엔 열리고 "사퇴" 쓴소리엔 열리지 않은 조국 수첩
10이상+
오전엔 환대,오후엔 질책…냉·온탕 오간 조국의 하루
100이상+
심상정 "개혁 위해 조국에 자기결단 요구할 수도"
황교안 이어 김문수·강효상도 릴레이 삭발…다음은 누구?
200이상+
500이상+
“백성 분노하면 임금도 뒤집는다” 강효상도 릴레이 삭발 동참
50이상+
강효상 삭발투쟁 동참..野 인사 6명째
200이상+
강효상도 삭발…"백성이 분노하면 임금도 뒤집을 수 있다"
'조국 사퇴' 野 릴레이 삭발…김문수·강효상 참여
200이상+
200이상+
[현장] 김문수 삭발 동참…"머리 깎고 의원직 던지자"
1,000이상+
김문수 "조국을 감옥으로" 눈물의 삭발식
50이상+
이언주·김문수 '눈물', 황교안 '눈 꼬옥~'…삭발의 기술
靑·외교부 잇딴 해명에도…꺼지지 않는 강경화-김현종 '불화설'
30이상+
100이상+
탈 많은 ‘김현종 스타일’ 어떻길래?…저돌적이고 성과 강조하는 터프가이
강경화-김현종 갈등 노출...靑 "확대해석인 듯"
10이상+
강경화 장관과 김현종 靑차장, 왜 싸웠나
파주 돼지열병에 9.19 평양공동선언 1주년 행사도 '비상'
10이상+
9·19평양선언 기념행사 취소도 검토…돼지열병에 

In [27]:
# 더보기 클릭한 후 기사 목록

for _ in after.select("div.cluster_text a"):
    print(_.text)

미래 먹거리 콘텐츠산업 육성…2022년까지 정책금융 1조 확대(종합)
30이상+
200이상+
文 “한류 생산유발 효과 20조…창작자 노력에 날개 달겠다”
1,600이상+
文대통령 "콘텐츠는 미래 먹거리…마음껏 상상하고 도전하라"
200이상+
"BTS·나영석 같은 창작자가 콘텐츠 강국 만들어"
"김정은 눈치 보는 정권" "젊은 청년 두 번 죽여”… 野, 하재헌 중사 공상 판정 맹비난
500이상+
300이상+
유승민 “북한의 보훈처냐…온 나라가 미쳐가고 있다”
400이상+
하재헌 중사 ‘공상’ 논란 정치권 확산…野 “관련자 문책해야”
300이상+
“대통령 정상 아니라서”… ‘북한 지뢰’ 공상 논란 목소리 높인 野
유성엽, 인사 온 조국에 “내려놓는 게 좋지 않겠나”…사퇴 요구
50이상+
30이상+
"검찰개혁"엔 열리고 "사퇴" 쓴소리엔 열리지 않은 조국 수첩
10이상+
오전엔 환대,오후엔 질책…냉·온탕 오간 조국의 하루
100이상+
심상정 "개혁 위해 조국에 자기결단 요구할 수도"
황교안 이어 김문수·강효상도 릴레이 삭발…다음은 누구?
200이상+
500이상+
“백성 분노하면 임금도 뒤집는다” 강효상도 릴레이 삭발 동참
50이상+
강효상 삭발투쟁 동참..野 인사 6명째
200이상+
강효상도 삭발…"백성이 분노하면 임금도 뒤집을 수 있다"
'조국 사퇴' 野 릴레이 삭발…김문수·강효상 참여
200이상+
200이상+
[현장] 김문수 삭발 동참…"머리 깎고 의원직 던지자"
1,000이상+
김문수 "조국을 감옥으로" 눈물의 삭발식
50이상+
이언주·김문수 '눈물', 황교안 '눈 꼬옥~'…삭발의 기술
靑·외교부 잇딴 해명에도…꺼지지 않는 강경화-김현종 '불화설'
30이상+
100이상+
탈 많은 ‘김현종 스타일’ 어떻길래?…저돌적이고 성과 강조하는 터프가이
강경화-김현종 갈등 노출...靑 "확대해석인 듯"
10이상+
강경화 장관과 김현종 靑차장, 왜 싸웠나
파주 돼지열병에 9.19 평양공동선언 1주년 행사도 '비상'
10이상+
9·19평양선언 기념행사 취소도 검토…돼지열병에 

In [21]:
driver.close()